In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import pickle
with open("data/images_l.pkl", 'rb') as f: labeled_images = pickle.load(f)
with open("data/labels_l.pkl", 'rb') as f: labels = pickle.load(f)
with open("data/images_ul.pkl", 'rb') as f: unlabeled_images = pickle.load(f)
with open("data/images_test.pkl", 'rb') as f: images_test = pickle.load(f)
print("Data loaded")

X_temp = np.append(labeled_images, unlabeled_images, axis=0)
X_temp = np.append(X_temp, images_test, axis=0)
mean = X_temp.mean()
std = X_temp.std()

del X_temp

labeled_images = (labeled_images - mean) / std
unlabeled_images = (unlabeled_images - mean) / std
images_test = (images_test - mean) / std
print("Data standardized")

label_dict = {}
for letter in range(0, 26):
    for number in range(0, 10):
        label = str(number) + chr(65 + letter)
        label_dict[letter*10 + number] = label
print(label_dict)

# Get the unique integer for a given label (used by the pytorch CNN architecture.)
def label_to_int(label):
    key_list = list(label_dict.keys())
    value_list = list(label_dict.values())
    key_index = value_list.index(label)
    return key_list[key_index]

# Get the label that corresponds with the given integer (must be between 0 and 259 inclusive.)
def int_to_label(int):
    return label_dict[int]

# The dataset of labels, as unique integers (i.e., size == 30,000)
def integer_labels():
    return arrmap(label_to_int, arrmap(binary_to_nl, labels))

# Shortcut function that maps a numpy array to a numpy array using the given lambda (fn.)
# I didn't want to keep writing this, though there is probably a better way to implement it... :/
def arrmap(fn, arr: np.array):
    return np.array(list(map(fn, arr)))

# Converts a single label into NUMBER/LETTER form.
# e.g., '[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]' --> '9D'
def binary_to_nl(binary_label):
    num = None
    letter = None
    for i in range(0, len(binary_label)):
        entry = binary_label[i]
        if i <= 9 and entry == 1:
            num = i
        elif entry == 1:
            letter = chr(64 - 9 + i)
    return str(num) + letter

# Converts back from a unique integer generated by the nl_to_int method into a number/letter label representation
def int_to_nl(int_label):
    num = 0
    for i in range(0, 10):
        if i * 100 <= int_label:
            num = i
    letter = None
    for i in range(0, 26):
        if (int_label - i) % 100 == 0:
            letter = chr(65 + i)
    return str(num) + letter

print("Data functions defined")

# Keys are integer labels, values are binary labels (original label)
binary_label_dict = {}
print(labels)
for binary_label in labels:
    integer_label = label_to_int(binary_to_nl(binary_label))
    binary_label_dict[integer_label] = binary_label
print(f'Number of inferred classes: {len(binary_label_dict)}')

# To parse the integer (0-259) --> Binary
def parsePrediction(prediction):
    binary_prediction = binary_label_dict[prediction]
    acc = ""
    for num in binary_prediction: 
        a = str(int(num))
        acc += a
    return acc

# To create a file to write to for submission
def parseForSubmission(predictions):    
    f = open("./submission.csv", "w")
    f.write(f"# Id,Category\n")
    for i in range(len(predictions)):
        f.write(f"{i},{str(parsePrediction(predictions[i]))}\n")
    f.close()

Data loaded
Data standardized
{0: '0A', 1: '1A', 2: '2A', 3: '3A', 4: '4A', 5: '5A', 6: '6A', 7: '7A', 8: '8A', 9: '9A', 10: '0B', 11: '1B', 12: '2B', 13: '3B', 14: '4B', 15: '5B', 16: '6B', 17: '7B', 18: '8B', 19: '9B', 20: '0C', 21: '1C', 22: '2C', 23: '3C', 24: '4C', 25: '5C', 26: '6C', 27: '7C', 28: '8C', 29: '9C', 30: '0D', 31: '1D', 32: '2D', 33: '3D', 34: '4D', 35: '5D', 36: '6D', 37: '7D', 38: '8D', 39: '9D', 40: '0E', 41: '1E', 42: '2E', 43: '3E', 44: '4E', 45: '5E', 46: '6E', 47: '7E', 48: '8E', 49: '9E', 50: '0F', 51: '1F', 52: '2F', 53: '3F', 54: '4F', 55: '5F', 56: '6F', 57: '7F', 58: '8F', 59: '9F', 60: '0G', 61: '1G', 62: '2G', 63: '3G', 64: '4G', 65: '5G', 66: '6G', 67: '7G', 68: '8G', 69: '9G', 70: '0H', 71: '1H', 72: '2H', 73: '3H', 74: '4H', 75: '5H', 76: '6H', 77: '7H', 78: '8H', 79: '9H', 80: '0I', 81: '1I', 82: '2I', 83: '3I', 84: '4I', 85: '5I', 86: '6I', 87: '7I', 88: '8I', 89: '9I', 90: '0J', 91: '1J', 92: '2J', 93: '3J', 94: '4J', 95: '5J', 96: '6J', 97: '7J',

In [4]:
import torch
import torch.nn as nn
def get_dataloaders(X, y, validation_split_index):
    '''Gets the training and validation dataloaders for a given dataset by using an index to split into training and validation.'''
    # get partitioned training and validation numpy arrays from the dataset
    x_train, x_val = data.split_train_and_val(X, validation_split_index)
    y_train, y_val = data.split_train_and_val(y, validation_split_index)

    # Make tensors out of the numpy arrays
    x_train, y_train, x_val, y_val = map(torch.tensor, (x_train, y_train, x_val, y_val))

    # Make TensorDatasets
    from torch.utils.data import TensorDataset
    train_ds = TensorDataset(x_train, y_train)
    val_ds = TensorDataset(x_val, y_val)

    # Make DataLoaders
    from torch.utils.data import DataLoader
    train_dl = DataLoader(train_ds, batch_size=32)
    val_dl = DataLoader(val_ds, batch_size=32)

    # return training and validation dataloaders :)
    return train_dl, val_dl

def downsample(batch, size=32):
    '''Downsamples the data to fit model requirements. I couldn't find a way around this that worked effectively...'''
    batch = batch.view(-1, 1, 56, 56) # Resize input to make it square
    downsample = nn.AdaptiveAvgPool2d(size) # Define a downsampling function
    batch = downsample(batch) # Downsample the batch's image data
    return batch

import assignment_data as data
from model_training import training_loop
def train_model(net, optimizer, criterion, epochs=2, device='cpu'):
    '''Loads data and trains a network on the image classification task.'''
    SPLIT_INDEX = 25000 # 25,000 training and 5,000 validation
    train_dl, val_dl = get_dataloaders(data.labeled_images, data.integer_labels(), SPLIT_INDEX)
    training_loop(net, criterion, optimizer, train_dl, val_dl, epochs, device)

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [6]:
import torch
from model_definitions import Net, LeNet5
from torch.nn import functional as F
net = Net()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
criterion = F.cross_entropy
train_model(Net(), optimizer, criterion)